# RCTs to Regression

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mixtape-Sessions/Machine-Learning/blob/main/Labs/python/RCT%20to%20Regression.ipynb)

## Gold standard: RCT

Treatment indicator: $D_i \in \left\{0,1\right\}$
> *example:* eligibility for expanded Medicaid

Outcome: $Y_i$
> *example:* number of doctor visits in past 6 months

Potential outcomes $Y_i(0),Y_i(1)$

Individual-level treatment effect $\delta _{i}=Y_{i}\left( 1\right) -Y_{i}\left( 0\right) $ (can never know
this).


Unbiased estimate of average treatment effect:
$$
\hat{\delta}=\bar{Y}_{1}-\bar{Y}_{0},
$$
or OLS coefficient on $D_{i}$ from this regression:
$$
Y_{i}=\alpha +\delta D_{i}+\varepsilon _{i}.
$$

Let's run it!

In [ ]:
# import some useful packages
import pandas as pd
# import numpy as np
from sklearn import linear_model

# read in data
oregonhie=pd.read_csv('https://github.com/Mixtape-Sessions/Machine-Learning/blob/main/Labs/data/oregon_hie_table5.csv?raw=true')

# data cleaning: drop observations with missing values in any of the variables we are going to need: 
regvarnames=['doc_num','treatment','weight']+[col for col in oregonhie if col.startswith('ddd')]
regvars=oregonhie[regvarnames].dropna()


# define outcome, treatment, weights, additional covariates
y=regvars['doc_num']
d=regvars[['treatment']]
w=regvars['weight']
x=regvars.drop(['doc_num','treatment','weight'],axis=1)

regvars

In [ ]:
# run weighted regression of outcome on treatment
lm=linear_model.LinearRegression()
lm.fit(d,y,w)

# display treatment effect
# display treatment effect
print("Estimated effect of Medicaid elibility on \n number of doctor visits" +
      " (bivariate): {:.3f}".format(lm.coef_[0]))

## Aluminum standard: Regression control

The bivariate regression above leans heavily on random assignment of treatment: 
$$
D_{i}\perp\!\!\!\!\perp \left( Y_{i}\left( 0\right) ,Y_{i}\left( 1\right) \right) .
$$
Sometimes, even in an RCT, treatment is assigned randomly only conditional on some set of covariates $X_i$. 
>*example:* in the Oregon HIE,
eligibility for Medicaid was granted via lottery, but households with more
members could have more lottery entries. So the lottery outcome is random
only conditional on household size. 

So what happens if we don't have random assignment? In terms of our regression model above, it means $\varepsilon_i$ may be correlated with $D_i$. For example, perhaps household size, $X_i$, which increases the probability of treatment, is also associated with more doctor visits. If $X_i$ is omitted from the model, it is part of the error term:
$$
\varepsilon_i=\beta X_i +\eta_i.
$$
We'll assume for now that everything else related to doctor visits ($\eta_i$) is unrelated to treatment. What does our bivariate regression coefficient deliver in this case?
$$
\hat{\delta}^{OLS}\underset{p}{\rightarrow}\frac{Cov\left(Y_i,D_i\right)}{Var\left(D_i\right)}=\delta+\gamma\frac{Cov\left(X_i,D_i\right)}{Var\left(D_i\right)}
$$
Simple regression gives us what we want ($\delta$) plus an **omitted variables bias** term. The form of this term tells us what kinds of $X_i$ variables we should take care to control for in our regressions.

According to the OVB formula, what kinds of variables should be be sure to control for in regressions?

Careful investigators will find a set of regressors $X_i$ for which they are willing to assume
treatment is as good as randomly assigned:
$$
D_i\perp\!\!\!\!\perp\left( Y_{i}\left( 0\right) ,Y_{i}\left( 1\right) \right) |X_{i}
\text{.}
$$
This combined with a linear model for the conditional expectation of $%
Y_{i}\left( 0\right) $ and $Y_{i}\left( 1\right) $ given $X_{i}$ means we
can estimate the average treatment via OLS on the following regression
equation:
$$
Y_{i}=\delta D_{i}+X_{i}^{\prime }\beta +\varepsilon _{i}.
$$


In [ ]:
# Add the household size indicators to our regressor set and run regression:
lm.fit(pd.concat([d,x],axis=1),y,w)
print("Estimated effect of Medicaid elibility on \n number of doctor visits" +
      " (with controls): {:.3f}".format(lm.coef_[0]))

How did the estimate of the effect of Medicaid eligility change? What does that tell us about the relationship between the included regressors and the outcome and treatment?

## Connection to ML

Where does machine learning fit into this? It might be tempting to treat
this regression as a prediction exercise where we are predicting $Y_{i}$
given $D_{i}$ and $X_{i}$. Don't give in to this temptation. We are not
after a prediction for $Y_{i}$, we are after a coefficient on $D_{i}$.
Modern machine learning algorithms are finely tuned for producing
predictions, but along the way they compromise coefficients. So how can we
deploy machine learning in the service of estimating the causal coefficient $\delta $?

To see where ML fits in, first remember that an equivalent way to estimate $%
\delta $ is the following three-step procedure:


1.   Regress $Y_{i}$ on $X_{i}$ and compute the residuals, $\tilde{Y}%
_{i}=Y_{i}-\hat{Y}_{i}^{OLS}$, where $\hat{Y}_{i}^{OLS}=X_{i}^{\prime
}\left( X^{\prime }X\right) ^{-1}X^{\prime }Y$
2.   Regress $D_{i}$ on $X_{i}$ and compute the residuals, $\tilde{D}%
_{i}=D_{i}-\hat{D}_{i}^{OLS}$, where $\hat{D}_{i}^{OLS}=X_{i}^{\prime
}\left( X^{\prime }X\right) ^{-1}X^{\prime }D$

3. Regress $\tilde{Y}_{i}$ on $\tilde{D}_{i}$.

Let's try it!

In [ ]:
# Regress outcome on covariates
yreg=linear_model.LinearRegression().fit(x,y,w)
# Calculate residuals
ytilde = y - yreg.predict(x)

# regress treatment on covariates
dreg = linear_model.LinearRegression().fit(x,d,w)
# Calculate residuals
dtilde = d - dreg.predict(x)

# regress ytilde on dtilde
lm.fit(dtilde,ytilde,w)
print("Estimated effect of Medicaid elibility on \n number of doctor visits" +
      " (partialled out): {:.3f}".format(lm.coef_[0]))

ML enters the picture by providing an alternate way to generate $\hat{Y}_i$ and $\hat{D}_i$ when OLS is not the best tool for the job. The first two steps are really just prediction exercises, and in principle any supervised machine learning algorithm can step in here. Back to the whiteboard!